In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class LoggedMappingMixin:

    __slots__ = ()

    def __getitem__(self, key):
        print(f'getting {key}')
        return super().__getitem__(key)

    def __setitem__(self, key, value):
        print(f'setting {key} {value}')
        return super().__setitem__(key, value)

    def __delitem__(self, key):
        print(f'deleting {key}')
        return super().__delitem__(key)


class SetOnceMappingMixin:

    __slots__ = ()

    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(f'{key} already set')

        super().__setitem__(key, value)


class StringKeysMappingMixin:

    __slots__ = ()

    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError(f'key must be str but got {type(key).__name__}')

        return super().__setitem__(key, value)


"""

>>> m = LoggedMappingMixin()
>>> m['a'] = 1
Traceback (most recent call last):
    ...
AttributeError: 'super' object has no attribute '__setitem__'

>>> class LoggedDict(LoggedMappingMixin, dict): pass
>>> d = LoggedDict()
>>> d['x'] = 23
setting x 23

>>> from collections import defaultdict
>>> class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict): pass
>>> d = SetOnceDefaultDict(list)
>>> d['x'].append(2)
>>> d['x'].append(3)
>>> d['x'] = 23
Traceback (most recent call last):
    ...
KeyError: 'x already set'

>>> from collections import OrderedDict
>>> class StringOrderedDict(StringKeysMappingMixin, SetOnceMappingMixin, OrderedDict): pass
>>> d = StringOrderedDict()
>>> d['x'] = 23
>>> d[42] = 10
Traceback (most recent call last):
    ...
TypeError: key must be str but got int
>>> d['x'] = 24
Traceback (most recent call last):
    ...
KeyError: 'x already set'
"""  # noqa: E501

doctest.testmod()

TestResults(failed=0, attempted=17)

In [4]:
class RestrictKeysMixin:

    def __init__(self, *args, _restrict_key_type, **kwargs):
        self.__restrict_key_type = _restrict_key_type
        super().__init__(*args, **kwargs)

    def __setitem__(self, key, value):
        if not isinstance(key, self.__restrict_key_type):
            raise TypeError(f'keys must be {self.__restrict_key_type!s}')

        super().__setitem__(key, value)


"""

>>> class RDict(RestrictKeysMixin, dict): pass
>>> d = RDict(_restrict_key_type=str)
>>> e = RDict([('name', 'Dave'), ('n', 37)], _restrict_key_type=str)
>>> f = RDict(name='Dave', n=37, _restrict_key_type=str)
>>> f
{'name': 'Dave', 'n': 37}
>>> f[42] = 10
Traceback (most recent call last):
    ...
TypeError: keys must be <class 'str'>
"""

doctest.testmod()

TestResults(failed=0, attempted=6)

In [5]:
def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__

    def __getitem__(self, key):
        print(f'getting {key}')
        return cls_getitem(self, key)

    def __setitem__(self, key, value):
        print(f'setting {key} {value}')
        return cls_setitem(self, key, value)

    def __delitem__(self, key):
        print(f'deleting {key}')
        return cls_delitem(self, key)

    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls


@LoggedMapping
class LoggedDict(dict):
    pass


d = LoggedDict()
d['x'] = 23
d['x'] = 1

setting x 23
setting x 1
